# <b>Introduction to Statistical Learning, 2nd Edition</b>
## by James, Witten, Hastie, Tibshirani

### Notes by Melis Tekant

## Chapter 6  - Linear Model Selection and Regularization

In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy.stats import multivariate_normal
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import random
import scipy.stats as st

Linear models are often quite competitive in real-world problems, and have the advantage of interpretability. These models can be improved by using fitting procedures other than least squares, such as subset selection, shrinkage, and dimension reduction.

Subset selection:

- Best subset selection: fit least squares regression for each combination of p predictors (leading to $2^p$ models) and identify the best. Do this by setting $M_0$ as the null model with no predictors, then for k = 1,2,...,p, fit all ${p \choose k}$ models with k predictors, pick the best among them (one which has the lowest RSS), and label it $M_k$. (For logistic regression, instead of RSS, $\textit{deviance} = -2ln(L)$ is used.) Then select the best $M_k$ using cross-validated prediction error, AIC, BIC, or adjusted R-squared. This way, the number of stored models reduces to p+1. This method is highly computationally intensive, so for large values of p, computational shortcuts (branch-and-bound techniques) can be used to eliminate some choices, or alternative methods that are more computationally efficient can be used. 

AIC ($\propto C_p$): Akaike information criterion. It is a relative estimator, where the lower the value is, the better the model is. It is often used for time series data. 
AIP $ = -2ln(L) + 2k$, where the $ln(L)$ term denotes the log-likelihood, and $k$ is the number of parameters, acting as a penalty for the AIC value.

(Note: this definition differs from the definition in the book: $C_p = \frac{1}{n}(RSS + 2k \hat \sigma^2)$, yet are equivalent given that the residuals are normally distributed.  

This can be shown via the following: Given the response $y = \omega X + \epsilon$, where $\epsilon \sim N(0,\sigma)$, the maxmimum likelihood is calculated by 

$$ L = \frac{1}{(2 \pi)^{N/2} \sigma^N} e^{-\frac{\sum(y-\omega X)^2}{2\sigma^2}}.$$

Therefore, $ln(L) = A + (-\frac{\sum(y-\omega X)^2}{2 \sigma^2})$, where A is a constant involving N and $\sigma$. 

Since $C_p$ is a relative estimator, it can be scaled, and constants can be added or removed from it, given that the process is done for each value that will be compared. 

Scaling the definition of $C_p$ in the text by $n/\hat \sigma^2$ gives equivalent results to the above definition with an additional constant and a scaling factor.)

BIC: Bayesian information criterion. Similar to AIC, but a different penalty term for complexity.
BIC $= -2ln(L) + ln(n)k$ $(\propto \frac{1}{n}(RSS+log(n)k\hat\sigma^2)$, where n is the number of data points. For n>7, this model places a larger penalty for k, so it typically selects for smaller number of predictors when compared to AIC. 

Adjusted R-squared: R-squared model that has been modified for the number of predictors, to penalize increasing k to prevent overfitting. 

$R^2 = 1- \frac{RSS}{TSS}$, where TSS = $\sum(y_i-\bar y)^2$.

Adjusted $R^2 = 1-\frac{RSS/(n-k-1)}{TSS/(n-1)}$.

Alternatively, test error can be estimated using validation set and cross-validation methods, which make fewer assumptions about the true model, and can be used even when $\sigma$ or k is hard to estimate.

In [47]:
credit = pd.read_csv('/Users/melistekant/Documents/Python Projects/ISLR2/Credit.csv')
credit.columns

Index(['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Own',
       'Student', 'Married', 'Region', 'Balance'],
      dtype='object')

In [48]:
# Separate Region into 2 dummy variables

dummy_south = np.zeros(len(credit.Region))
dummy_west = np.zeros(len(credit.Region))
dummy_south[credit.Region == 'South'] = 1 
dummy_west[credit.Region == 'West'] = 1
credit2 = credit.drop(columns = 'Region', axis =1)
credit2['West'] = dummy_west.astype('int')
credit2['South'] = dummy_south.astype('int')

yndict = {'Yes':1, 'No': 0}
credit2[['Own','Student','Married']] = credit2[{'Own','Student','Married'}].applymap(lambda x: yndict[x])
credit2.head()

,Income,Limit,Rating,Cards,Age,Education,Own,Student,Married,Balance,West,South
0,14.891,3606,283,2,34,11,1,0,0,333,0,1
1,106.025,6645,483,3,82,15,1,1,1,903,1,0
2,104.593,7075,514,4,71,11,0,0,0,580,1,0
3,148.924,9504,681,3,36,11,0,0,1,964,1,0
4,55.882,4897,357,2,68,16,1,0,0,331,0,1


In [50]:
y = credit2.Balance
X = credit2.drop(columns = 'Balance',axis=1)
knum = np.arange(0,len(credit2.columns))




array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [17]:
credit.Region.unique()

array(['South', 'West', 'East'], dtype=object)

- Stepwise selection: For large p where best subset selection might not be usable, stepwise methods, which test much more restricted models, can be used.
    
     Forward stepwise selection: Start with no predictors ($M_0$) and add one predictor at a time (the one that gives the greatest additional improvement to the fit, measured by lowest RSS or highest R-squared, call $M_k$ for k predictors in the model) until all are in the model. This leads to $1+p(p+1)/2$ fits instead of $2^p$. Select the best $M_k$ using cross-validated prediction error, AIC, BIC, or adjusted R-squared. 
     
     Backward stepwise selection: Similar to forward stewpsie selection, but start with all predictors, and remove predictors one at a time, selecting the best k-1 predictors (given by lowest RSS or highest R-squared value) at each step, then find best $M_k$ by one of the metrics outlined above. This method requires n>p, while forward stepwise selection does not. 
     
     Hybrid approaches: Predictors are added to the model, but can also be removed, if they do not provide improvement in the new fit. This approach tries to marry the subset selection model with the computational advantages of stepwise selection methods. 

In [2]:
# Code for Table 6.1 goes here.

If the error vs. k graph is quite flat, use the 'one-standard-error rule': calculate standard error and estimate MSE for each k. Select the simplest (smallest k) model for which test error is within one standard error of the lowest error value. 